<b> <font size="+2"> Calculate pseudobulk expression </b> </font> <br>
by summing the counts 

Date: 2023-11-20 <br>
Author: ASF, SR

# Load libraries

In [1]:
suppressPackageStartupMessages({
library(vroom)
library(tidyverse)
})

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ readr::col_character()   masks vroom::col_character()
✖ readr::col_date()        masks vroom::col_date()
✖ readr::col_datetime()    masks vroom::col_datetime()
✖ readr::col_double()      masks vroom::col_double()
✖ readr::col_factor()      masks vroom::col_factor()
✖ readr::col_guess()       masks vroom::col_guess()
✖ readr::col_integer()     masks vroom::col_integer()
✖ readr::col_logical()     masks vroom::col_logical()
✖ readr::col_number()      masks vroom::col_number()
✖ readr::col_skip()        masks vroom::col_skip()
✖ readr::col_time()        masks vroom::col_time()
✖ readr::cols()            masks vroom::cols()
✖ readr::date_names_lang(

# Prep

In [ ]:
# data needs to be processed in chuncks due to RAM 
# vroom can be told which lines to read

In [2]:
# read in metadata
obs=vroom::vroom("/psycl/g/mpsagbinder/AnnaFroehlich/sn_postmortem/datasets/adata.obs_20231120.tsv", delim="\t", col_names = TRUE)

Rows: 787685 Columns: 71
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr (29): index, sample, region, experiment, AgeBin, Sex, Ethnic.origin, Cla...
dbl (38): individual, batch, SU.Number, Age, Status, Trauma.Code, PMI, Brain...
num  (4): leiden_r0.5_sub7, leiden_r0.5_sub3, leiden_r0.5_sub15, leiden_r0.5...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
# read in the gene information
var=vroom::vroom("/psycl/g/mpsagbinder/AnnaFroehlich/sn_postmortem/datasets/adata.var_20231120.tsv", delim="\t", col_names = TRUE)

New names:
• `` -> `...1`
Rows: 26195 Columns: 14
── Column specification ────────────────────────────────────────────────────────
Delimiter: "\t"
chr  (2): ...1, gene_ids
dbl (10): mean_counts, log1p_mean_counts, pct_dropout_by_counts, total_count...
lgl  (2): mito, highly_variable

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [4]:
# Compute the lengths and values of runs of equal values in a vector
rle=rle(obs$individual)
#rle
#length(rle$lengths)

# vector with number of nuclei per individual
l=rle$lengths
l

[1] 10419  9759  8858  7654  9159  8218 10087  3800  8417 11427  6779  4275
[13]  5747  6533  6494  6168  6652  5592  6274  6535 10100 13573  4997 10186
[25]  9352 10366 11121  4455  8181  8722  9067  9706  7274 12185  8871 10421
[37] 12657  8615  8958  3903  7863  6874  9414 11739  7292 10846  8774 13915
[49]  5714 11794  8907  9604  8080  9958  4042  9327  9881  9083 12193 12715
[61] 12514  9872 14216  8600 11817  6501 12336  4276 15719  6241  8658 11640
[73]  4015  5534 10509  8786 14193  6614 13426  9128  9812 10940 11983  7598
[85]  7629 12691 12865

In [5]:
# vector with identifiers of individuals
v=rle$values
#v

In [6]:
# unlist() function is used to convert a list to vector
# make a vector containg the number of nuclei per individual summed up step by step
e=unlist(lapply(1:length(rle$lengths), function(x){
    sum(l[1:x])
}))
e

[1]  10419  20178  29036  36690  45849  54067  64154  67954  76371  87798
[11]  94577  98852 104599 111132 117626 123794 130446 136038 142312 148847
[21] 158947 172520 177517 187703 197055 207421 218542 222997 231178 239900
[31] 248967 258673 265947 278132 287003 297424 310081 318696 327654 331557
[41] 339420 346294 355708 367447 374739 385585 394359 408274 413988 425782
[51] 434689 444293 452373 462331 466373 475700 485581 494664 506857 519572
[61] 532086 541958 556174 564774 576591 583092 595428 599704 615423 621664
[71] 630322 641962 645977 651511 662020 670806 684999 691613 705039 714167
[81] 723979 734919 746902 754500 762129 774820 787685

In [7]:
#to later determine how many lines to skip in the loop
s=e-l
s

[1]      0  10419  20178  29036  36690  45849  54067  64154  67954  76371
[11]  87798  94577  98852 104599 111132 117626 123794 130446 136038 142312
[21] 148847 158947 172520 177517 187703 197055 207421 218542 222997 231178
[31] 239900 248967 258673 265947 278132 287003 297424 310081 318696 327654
[41] 331557 339420 346294 355708 367447 374739 385585 394359 408274 413988
[51] 425782 434689 444293 452373 462331 466373 475700 485581 494664 506857
[61] 519572 532086 541958 556174 564774 576591 583092 595428 599704 615423
[71] 621664 630322 641962 645977 651511 662020 670806 684999 691613 705039
[81] 714167 723979 734919 746902 754500 762129 774820

# compute pseudobulk

<b> Explanantion </b>
1) counts - get the count matrix of the first individual (from the knowledge how many rows belong to each individual) (go through the individuals 1:87 (1:length(e))
2) clusters_chunk - extract the cluster assignment of the rows belonging to the current individual  
3) clusters - get the unique clusters (observations)
4) res - go through the clusters of the current individual <br>
   print the name of the current cluster <br>
   get the index/indices of the lines with the current cluster assignment <br>
   if this is more than 1, then the cluster has more than one cell assigned to it and the sum of all cells in this cluster will be calculated <br>
   if the index is 1 then this cluster only has one cell and you simply take its count value
5) do.call('rbind) Makes one data.table

calculate the <b> sum </b> of all counts using <b> colSums() </b> <br>

<b> if else condition: </b> <br>
<b> else </b>: when the cluster only has one cell then just take the count values of this cell <br>
<b> if </b>: if the cluster has more than 1 cell (idx >1) then calculate the sum with colSums()

In [ ]:
# roughly takes 1 h 45 min

In [8]:
pseudoBulk_counts_chunks <- lapply(1:length(e) , function(x) {
  
  print(paste("Chunk", x, "out of", length(e)))
  counts= vroom::vroom("/psycl/g/mpsagbinder/AnnaFroehlich/sn_postmortem/datasets/adata.rawcounts_20231120.tsv", 
                       delim="\t", 
                       col_names = var$gene_ids, 
                       skip = s[x], 
                       n_max = e[x]-s[x], 
                       show_col_types = FALSE ) %>% as.matrix()
  clusters_chunk=obs[["celltypes_final"]][c((s[x]+1):e[x])]
  clusters=unique(clusters_chunk)
  res <- lapply(clusters, function(y){ 
    print(y)
    idx=which(clusters_chunk==y)
    if(length(idx)>1){
      dplyr::tibble(
        sum_counts = colSums(counts[idx,]),
        gene = colnames(counts),
        cluster = y,
        sample = v[x]
      )
    } else {
      dplyr::tibble(
        sum_counts = counts[idx,],
        gene = colnames(counts),
        cluster = y,
        sample = v[x]
      )
    }
  })
  res <- do.call('rbind', res)
  gc()
  return(res)
  
})

[1] "Chunk 1 out of 87"
[1] "Oligodendrocyte"
[1] "In_PVALB_Ba"
[1] "Exc_L4-6_3"
[1] "Exc_L2-3"
[1] "In_LAMP5_1"
[1] "Astro_PP"
[1] "In_VIP"
[1] "Exc_L4-6_1"
[1] "OPC"
[1] "Exc_L5-6_1"
[1] "In_SST"
[1] "Exc_L4-6_2"
[1] "In_LAMP5_2"
[1] "Microglia"
[1] "Exc_L5-6_HTR2C"
[1] "In_RELN"
[1] "Astro_FB"
[1] "Exc_L3-5"
[1] "In_PVALB_Ch"
[1] "Endothelial"
[1] "Exc_L5-6_2"
[1] "Chunk 2 out of 87"
[1] "Microglia"
[1] "Exc_L2-3"
[1] "In_LAMP5_1"
[1] "Exc_L5-6_1"
[1] "In_RELN"
[1] "OPC"
[1] "Astro_FB"
[1] "In_SST"
[1] "Oligodendrocyte"
[1] "In_VIP"
[1] "Exc_L4-6_3"
[1] "In_PVALB_Ba"
[1] "Endothelial"
[1] "Exc_L4-6_2"
[1] "Exc_L4-6_1"
[1] "Exc_L3-5"
[1] "Astro_PP"
[1] "In_PVALB_Ch"
[1] "In_LAMP5_2"
[1] "Exc_L5-6_2"
[1] "Exc_L5-6_HTR2C"
[1] "Chunk 3 out of 87"
[1] "In_PVALB_Ba"
[1] "Astro_FB"
[1] "Exc_L4-6_2"
[1] "Astro_PP"
[1] "Exc_L2-3"
[1] "In_VIP"
[1] "Oligodendrocyte"
[1] "Microglia"
[1] "Exc_L4-6_1"
[1] "OPC"
[1] "Exc_L5-6_HTR2C"
[1] "In_PVALB_Ch"
[1] "In_RELN"
[1] "In_SST"
[1] "Exc_L3-5"
[1] "

In [10]:
#make one big table containing all individuals and all celltypes
pseudoBulk_counts <- do.call("rbind", pseudoBulk_counts_chunks) 

In [11]:
# save object
save(pseudoBulk_counts, file="../datasets/PseudoBulkFinal/ALL_pseudoBulk_counts.Rda")

In [12]:
# save by cluster
# spread to matrix
# make sure to put the correct file path

clusters=unique(pseudoBulk_counts[["cluster"]])
lapply(clusters, function(x) {
  subset(pseudoBulk_counts, cluster==x) %>% 
    spread(key=gene, value=sum_counts) %>% 
    write.table(file=paste0( "../datasets/PseudoBulkFinal/",x,"_pseudobulk_counts_matrix.tsv"), sep="\t", quote=F, col.names=T, row.names=F)
  } )

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

[[14]]
NULL

[[15]]
NULL

[[16]]
NULL

[[17]]
NULL

[[18]]
NULL

[[19]]
NULL

[[20]]
NULL

[[21]]
NULL

In [13]:
sessionInfo()

R version 4.3.1 (2023-06-16)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: CentOS Linux 7 (Core)

Matrix products: default
BLAS/LAPACK: /psycl/u/afroe/conda-envs/dreamlet_duplicated/lib/libopenblasp-r0.3.24.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Berlin
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lubridate_1.9.3 forcats_1.0.0   stringr_1.5.0   dplyr_1.1.3    
 [5] purrr_1.0.2     readr_2.1.4     tidyr_1.3.0     tibble_3.2.1   
 [9] ggplot2_3.4.4   tidyverse_2.0.0 vroom_1.6.4    

loaded via a namesp